#New code

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import NearestNeighbors
from sklearn.utils.class_weight import compute_class_weight
from collections import Counter
import requests
import zipfile
import os
import time
import psutil
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

class ActivityRecognitionSystem:


    def __init__(self, window_size=100, overlap=0.5):
        self.window_size = window_size
        self.overlap = overlap
        self.step_size = int(window_size * (1 - overlap))
        self.scaler = StandardScaler()
        self.label_encoder = LabelEncoder()
        self.model = None
        self.history = None

        # Activity mapping from the dataset documentation
        self.activity_map = {
            1: 'walking',
            2: 'descending_stairs',
            3: 'ascending_stairs',
            4: 'driving',
            77: 'clapping',
            99: 'non_study_activity'
        }

    def download_and_extract_dataset(self):

        base_path = '/content/accelerometry_data'
        expected_path = f'{base_path}/labeled-raw-accelerometry-data-captured-during-walking-stair-climbing-and-driving-1.0.0/raw_accelerometry_data'

        # Check if dataset already exists
        if os.path.exists(expected_path):
            print("Dataset already exists, skipping download.")
            return True

        print("Dataset not found. Downloading PhysioNet accelerometry dataset...")

        try:
            # Create base directory
            os.makedirs(base_path, exist_ok=True)

            # Download URL
            url = "https://physionet.org/static/published-projects/accelerometry-walk-climb-drive/accelerometry-walk-climb-drive-1.0.0.zip"
            zip_path = f'{base_path}/dataset.zip'

            # Download with progress
            print("Downloading dataset (this may take a few minutes)...")
            response = requests.get(url, stream=True)
            response.raise_for_status()

            total_size = int(response.headers.get('content-length', 0))
            downloaded_size = 0

            with open(zip_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
                        downloaded_size += len(chunk)
                        if total_size > 0:
                            progress = (downloaded_size / total_size) * 100
                            print(f"\rProgress: {progress:.1f}%", end='')

            print("\nDownload complete! Extracting...")

            # Extract the zip file
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(base_path)

            # Remove the zip file to save space
            os.remove(zip_path)

            print("Dataset extracted successfully!")
            return True

        except Exception as e:
            print(f"Error downloading dataset: {e}")
            print("Please download manually from: https://physionet.org/content/accelerometry-walk-climb-drive/1.0.0/")
            return False

    def load_data(self):

        # Try to download dataset if not present
        if not self.download_and_extract_dataset():
            return None

        print("Loading accelerometry data from dataset...")

        # Based on the PhysioNet dataset structure
        data_dir = '/content/accelerometry_data/labeled-raw-accelerometry-data-captured-during-walking-stair-climbing-and-driving-1.0.0/raw_accelerometry_data'

        if not os.path.exists(data_dir):
            print(f"Error: Directory {data_dir} not found even after download!")
            # Try alternative path in case of extraction differences
            alt_path = '/content/accelerometry_data/accelerometry-walk-climb-drive-1.0.0/raw_accelerometry_data'
            if os.path.exists(alt_path):
                data_dir = alt_path
                print(f"Found dataset at alternative path: {data_dir}")
            else:
                print("Dataset structure check:")
                if os.path.exists('/content/accelerometry_data'):
                    for item in os.listdir('/content/accelerometry_data'):
                        print(f"  - {item}")
                return None

        all_data = []
        participant_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

        print(f"Found {len(participant_files)} participant files")

        # Use all participants for better results
        for file in participant_files:
            file_path = os.path.join(data_dir, file)
            try:
                df = pd.read_csv(file_path)
                df['participant'] = file.replace('.csv', '')
                all_data.append(df)
                print(f"Loaded {file}: {len(df)} samples")
            except Exception as e:
                print(f"Error loading {file}: {e}")

        if not all_data:
            print("No data loaded!")
            return None

        combined_data = pd.concat(all_data, ignore_index=True)
        print(f"Total samples loaded: {len(combined_data)}")
        print(f"Columns: {list(combined_data.columns)}")

        return combined_data

    def preprocess_data(self, data):
        """
        Preprocess the accelerometry data

        Following guidelines: Pay attention to pre-processing phase (normalize data)
        """
        print("Preprocessing data...")

        # Filter out clapping and non-study activities for main classification
        # Focus on main activities: walking, stairs, driving
        main_activities = data[data['activity'].isin([1, 2, 3, 4])].copy()

        print("Activity distribution:")
        activity_counts = main_activities['activity'].value_counts().sort_index()
        for activity_code, count in activity_counts.items():
            activity_name = self.activity_map[activity_code]
            percentage = (count / len(main_activities)) * 100
            print(f"  {activity_name}: {count} samples ({percentage:.1f}%)")

        # Extract accelerometer columns (12 features: 4 sensors × 3 axes)
        accel_columns = ['lw_x', 'lw_y', 'lw_z',  # left wrist
                        'lh_x', 'lh_y', 'lh_z',   # left hip
                        'la_x', 'la_y', 'la_z',   # left ankle
                        'ra_x', 'ra_y', 'ra_z']   # right ankle

        # Check for missing values
        missing_data = main_activities[accel_columns].isnull().sum().sum()
        if missing_data > 0:
            print(f"Warning: {missing_data} missing values found. Filling with forward fill.")
            main_activities[accel_columns] = main_activities[accel_columns].fillna(method='ffill')

        # Normalize the accelerometer data (critical preprocessing step)
        print("Normalizing accelerometer data...")
        main_activities[accel_columns] = self.scaler.fit_transform(main_activities[accel_columns])

        # Encode activity labels
        main_activities['activity_encoded'] = self.label_encoder.fit_transform(main_activities['activity'])

        return main_activities, accel_columns

    def create_windows(self, data, accel_columns):
        print(f"Creating windows (size={self.window_size}, step={self.step_size})...")

        X_windows = []
        y_windows = []
        participant_ids = []

        # Process each participant separately
        for participant in data['participant'].unique():
            participant_data = data[data['participant'] == participant].reset_index(drop=True)

            # Create windows for this participant
            for i in range(0, len(participant_data) - self.window_size + 1, self.step_size):
                window_data = participant_data.iloc[i:i + self.window_size]

                # Check if the window contains only one activity
                if window_data['activity'].nunique() == 1:
                    window_features = window_data[accel_columns].values
                    activity_label = window_data['activity_encoded'].iloc[0]

                    X_windows.append(window_features)
                    y_windows.append(activity_label)
                    participant_ids.append(participant)

        X = np.array(X_windows)
        y = np.array(y_windows)
        participant_ids = np.array(participant_ids)

        print(f"Created {len(X)} windows from {len(np.unique(participant_ids))} participants")
        print(f"Window shape: {X.shape}")

        # RETURN THREE VALUES
        return X, y, participant_ids

    def apply_data_balancing(self, X, y, balancing_method='smote'):

        print(f"\nApplying {balancing_method} balancing...")

        # Print original distribution
        original_counts = Counter(y)
        print("Original class distribution:")
        for class_idx, count in sorted(original_counts.items()):
            activity_name = self.activity_map[self.label_encoder.classes_[class_idx]]
            percentage = (count / len(y)) * 100
            print(f"  {activity_name}: {count} samples ({percentage:.1f}%)")

        if balancing_method == 'smote':
            return self._apply_smote(X, y)
        elif balancing_method == 'undersampling':
            return self._apply_undersampling(X, y)
        elif balancing_method == 'oversampling':
            return self._apply_oversampling(X, y)
        elif balancing_method == 'class_weights':
            return self._compute_class_weights(y), X, y
        else:
            raise ValueError("balancing_method must be 'smote', 'undersampling', 'oversampling', or 'class_weights'")

    def _apply_smote(self, X, y):

        print("Applying lightweight SMOTE")

        class_counts = Counter(y)

        # Set a reasonable target count (median * 1.2 to be more conservative)
        target_count = int(np.median(list(class_counts.values())) * 1.2)
        print(f"Target samples per class: {target_count}")

        X_balanced = []
        y_balanced = []

        for class_idx in np.unique(y):
            class_mask = y == class_idx
            class_samples = X[class_mask]
            class_labels = y[class_mask]

            current_count = len(class_samples)
            activity_name = self.activity_map[self.label_encoder.classes_[class_idx]]

            if current_count < target_count:
                # Minority class - oversample with SMOTE but limit synthetic data to 25%
                samples_needed = target_count - current_count

                # Calculate maximum synthetic samples (25% of final count)
                max_synthetic = int(target_count * 0.25)

                # Use the minimum of what's needed and what's allowed (25% rule)
                synthetic_to_create = min(samples_needed, max_synthetic)

                # If we can't reach target with 25% synthetic, adjust target downward
                if synthetic_to_create < samples_needed:
                    adjusted_target = current_count + synthetic_to_create
                    print(f"  {activity_name}: Adjusting target from {target_count} to {adjusted_target} due to 25% synthetic limit")
                    target_count_for_class = adjusted_target
                else:
                    target_count_for_class = target_count

                # Create synthetic samples
                synthetic_samples = []
                for _ in range(synthetic_to_create):
                    # Pick two random samples from the same class
                    idx1, idx2 = np.random.choice(current_count, 2, replace=False if current_count > 1 else True)
                    sample1 = class_samples[idx1]
                    sample2 = class_samples[idx2]

                    # Linear interpolation with random alpha
                    alpha = np.random.random()
                    synthetic_sample = sample1 + alpha * (sample2 - sample1)
                    synthetic_samples.append(synthetic_sample)

                # Add original samples
                X_balanced.extend(class_samples)
                y_balanced.extend(class_labels)

                # Add synthetic samples
                if synthetic_samples:
                    X_balanced.extend(synthetic_samples)
                    y_balanced.extend([class_idx] * synthetic_to_create)

                synthetic_percentage = (synthetic_to_create / target_count_for_class) * 100
                print(f"  {activity_name}: {current_count} → {target_count_for_class} (+{synthetic_to_create} synthetic, {synthetic_percentage:.1f}%)")

            elif current_count > target_count:
                # Majority class - undersample
                indices = np.random.choice(current_count, target_count, replace=False)
                sampled_X = class_samples[indices]
                sampled_y = class_labels[indices]

                X_balanced.extend(sampled_X)
                y_balanced.extend(sampled_y)

                print(f"  {activity_name}: {current_count} → {target_count} (-{current_count - target_count} removed)")

            else:
                # Already at target
                X_balanced.extend(class_samples)
                y_balanced.extend(class_labels)
                print(f"  {activity_name}: {current_count} samples (no change)")

        X_balanced = np.array(X_balanced)
        y_balanced = np.array(y_balanced)

        # Shuffle
        indices = np.random.permutation(len(X_balanced))
        return None, X_balanced[indices], y_balanced[indices]

    def _apply_undersampling(self, X, y):

        print("Applying random undersampling...")

        class_counts = Counter(y)
        min_count = min(class_counts.values())

        X_balanced = []
        y_balanced = []

        for class_idx in np.unique(y):
            class_mask = y == class_idx
            class_samples = X[class_mask]
            class_labels = y[class_mask]

            if len(class_samples) > min_count:
                indices = np.random.choice(len(class_samples), min_count, replace=False)
                sampled_X = class_samples[indices]
                sampled_y = class_labels[indices]
            else:
                sampled_X = class_samples
                sampled_y = class_labels

            X_balanced.append(sampled_X)
            y_balanced.append(sampled_y)

            activity_name = self.activity_map[self.label_encoder.classes_[class_idx]]
            print(f"  {activity_name}: {len(class_samples)} → {len(sampled_X)} samples")

        X_balanced = np.vstack(X_balanced)
        y_balanced = np.concatenate(y_balanced)

        indices = np.random.permutation(len(X_balanced))
        return None, X_balanced[indices], y_balanced[indices]

    def _apply_oversampling(self, X, y):

        print("Applying random oversampling...")

        class_counts = Counter(y)
        max_count = max(class_counts.values())

        X_balanced = []
        y_balanced = []

        for class_idx in np.unique(y):
            class_mask = y == class_idx
            class_samples = X[class_mask]
            class_labels = y[class_mask]

            if len(class_samples) < max_count:
                indices = np.random.choice(len(class_samples), max_count, replace=True)
                sampled_X = class_samples[indices]
                sampled_y = class_labels[indices]
            else:
                sampled_X = class_samples
                sampled_y = class_labels

            X_balanced.append(sampled_X)
            y_balanced.append(sampled_y)

            activity_name = self.activity_map[self.label_encoder.classes_[class_idx]]
            print(f"  {activity_name}: {len(class_samples)} → {len(sampled_X)} samples")

        X_balanced = np.vstack(X_balanced)
        y_balanced = np.concatenate(y_balanced)

        indices = np.random.permutation(len(X_balanced))
        return None, X_balanced[indices], y_balanced[indices]

    def _compute_class_weights(self, y):

        print("Computing class weights for balanced training...")

        classes = np.unique(y)
        class_weights = compute_class_weight('balanced', classes=classes, y=y)
        class_weight_dict = dict(zip(classes, class_weights))

        print("Class weights:")
        for class_idx, weight in class_weight_dict.items():
            activity_name = self.activity_map[self.label_encoder.classes_[class_idx]]
            print(f"  {activity_name}: {weight:.3f}")

        return class_weight_dict

    def create_cnn_model(self, input_shape, num_classes):

        print("Creating CNN model with anti-overfitting measures...")

        model = tf.keras.Sequential([
            # Reduced complexity - fewer filters
            tf.keras.layers.Conv1D(filters=16, kernel_size=5, activation='relu', input_shape=input_shape),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            tf.keras.layers.Dropout(0.4),  # Increased dropout

            tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            tf.keras.layers.Dropout(0.5),  # Increased dropout

            tf.keras.layers.GlobalMaxPooling1D(),
            tf.keras.layers.Dropout(0.6),  # High dropout before dense layers

            # Smaller dense layers
            tf.keras.layers.Dense(32, activation='relu'),
            tf.keras.layers.Dropout(0.7),
            tf.keras.layers.Dense(num_classes, activation='softmax')
        ])

        return model

    def create_lstm_model(self, input_shape, num_classes):

        print("Creating LSTM model with anti-overfitting measures...")

        model = tf.keras.Sequential([
            # Smaller LSTM units
            tf.keras.layers.LSTM(32, return_sequences=True, input_shape=input_shape,
                               dropout=0.3, recurrent_dropout=0.3),
            tf.keras.layers.BatchNormalization(),

            tf.keras.layers.LSTM(16, return_sequences=False,
                               dropout=0.4, recurrent_dropout=0.4),
            tf.keras.layers.Dropout(0.6),

            # Smaller dense layer
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dropout(0.7),
            tf.keras.layers.Dense(num_classes, activation='softmax')
        ])

        return model

    def create_hybrid_model(self, input_shape, num_classes):

        print("Creating Hybrid model with anti-overfitting measures...")

        model = tf.keras.Sequential([
            # Smaller CNN part
            tf.keras.layers.Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=input_shape),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling1D(pool_size=2),
            tf.keras.layers.Dropout(0.4),

            # Smaller LSTM part
            tf.keras.layers.LSTM(16, return_sequences=False, dropout=0.3, recurrent_dropout=0.3),
            tf.keras.layers.Dropout(0.6),

            # Smaller dense layer
            tf.keras.layers.Dense(16, activation='relu'),
            tf.keras.layers.Dropout(0.7),
            tf.keras.layers.Dense(num_classes, activation='softmax')
        ])

        return model

    def train_model(self, X_train, X_test, y_train, y_test, model_type='cnn', class_weights=None):

        print(f"\nTraining {model_type.upper()} model...")

        num_classes = len(np.unique(y_train)) # Get the actual number of unique classes in the training data

        if model_type == 'cnn':
            self.model = self.create_cnn_model(X_train.shape[1:], num_classes)
        elif model_type == 'lstm':
            self.model = self.create_lstm_model(X_train.shape[1:], num_classes)
        elif model_type == 'hybrid':
            self.model = self.create_hybrid_model(X_train.shape[1:], num_classes)

        self.model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        callbacks = [
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7)
        ]

        start_time = time.time()
        start_memory = psutil.Process().memory_info().rss / 1024 / 1024

        self.history = self.model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=30,  # Reduced for faster testing
            batch_size=32,
            callbacks=callbacks,
            class_weight=class_weights,
            verbose=0
        )

        training_time = time.time() - start_time
        end_memory = psutil.Process().memory_info().rss / 1024 / 1024
        memory_usage = end_memory - start_memory

        return training_time, memory_usage

    def evaluate_model(self, X_test, y_test):

        y_pred_proba = self.model.predict(X_test)
        y_pred = np.argmax(y_pred_proba, axis=1)
        accuracy = accuracy_score(y_test, y_pred)

        return accuracy, y_pred, y_pred_proba

def participant_aware_split(X, y, participant_ids, test_size=0.2, random_state=42):

    print("Performing participant-aware train/test split...")

    unique_participants = np.unique(participant_ids)
    np.random.seed(random_state)
    np.random.shuffle(unique_participants)

    # Split participants, not windows
    n_test_participants = max(1, int(len(unique_participants) * test_size))
    test_participants = set(unique_participants[:n_test_participants])
    train_participants = set(unique_participants[n_test_participants:])

    print(f"Train participants: {len(train_participants)}")
    print(f"Test participants: {len(test_participants)}")

    # Create masks
    train_mask = np.array([p in train_participants for p in participant_ids])
    test_mask = np.array([p in test_participants for p in participant_ids])

    X_train, X_test = X[train_mask], X[test_mask]
    y_train, y_test = y[train_mask], y[test_mask]

    print(f"Train windows: {len(X_train)}")
    print(f"Test windows: {len(X_test)}")

    return X_train, X_test, y_train, y_test

def compare_architectures():

    print("="*60)
    print("ACTIVITY RECOGNITION - FAST MODE (30% DATASET)")
    print("="*60)

    total_start_time = time.time()

    # Initialize system with smaller windows for speed
    system = ActivityRecognitionSystem(window_size=25, overlap=0.1)  # Much smaller windows, minimal overlap

    print("Step 1/6: Loading dataset...")
    step_start = time.time()
    raw_data = system.load_data()
    if raw_data is None:
        return
    print(f"✓ Dataset loaded in {time.time() - step_start:.1f}s")

    print("\nStep 2/6: Preprocessing data...")
    step_start = time.time()
    processed_data, accel_columns = system.preprocess_data(raw_data)
    print(f"✓ Preprocessing completed in {time.time() - step_start:.1f}s")

    print("\nStep 3/6: Creating windows...")
    step_start = time.time()
    X, y, participant_ids = system.create_windows(processed_data, accel_columns)
    print(f"✓ Window creation completed in {time.time() - step_start:.1f}s")

    print("\nStep 4/6: Splitting data...")
    step_start = time.time()
    X_train_orig, X_test, y_train_orig, y_test = participant_aware_split(
        X, y, participant_ids, test_size=0.2, random_state=42)
    print(f"✓ Data split completed in {time.time() - step_start:.1f}s")

    print(f"\nDataset sizes:")
    print(f"  Training: {X_train_orig.shape[0]} samples")
    print(f"  Test: {X_test.shape[0]} samples")

    balancing_methods = ['smote', 'class_weights']
    models_to_compare = ['cnn', 'hybrid']
    total_combinations = len(balancing_methods) * len(models_to_compare)
    current_combination = 0

    all_results = {}

    print(f"\nStep 5/6: Training {total_combinations} model combinations...")

    for balancing_method in balancing_methods:
        print(f"\n{'='*50}")
        print(f"BALANCING METHOD: {balancing_method.upper()}")
        print(f"{'='*50}")

        step_start = time.time()
        if balancing_method == 'class_weights':
            class_weights, X_train, y_train = system.apply_data_balancing(
                X_train_orig, y_train_orig, balancing_method)
        else:
            _, X_train, y_train = system.apply_data_balancing(
                X_train_orig, y_train_orig, balancing_method)
            class_weights = None
        print(f"✓ Balancing completed in {time.time() - step_start:.1f}s")

        method_results = {}

        for model_type in models_to_compare:
            current_combination += 1
            print(f"\n[{current_combination}/{total_combinations}] Training {model_type.upper()} with {balancing_method}")

            model_system = ActivityRecognitionSystem(window_size=25, overlap=0.1)
            model_system.scaler = system.scaler
            model_system.label_encoder = system.label_encoder

            model_start_time = time.time()
            training_time, memory_usage = model_system.train_model(
                X_train, X_test, y_train, y_test,
                model_type=model_type, class_weights=class_weights)

            accuracy, y_pred, y_pred_proba = model_system.evaluate_model(X_test, y_test)

            method_results[model_type] = {
                'accuracy': accuracy,
                'training_time': training_time,
                'memory_usage': memory_usage
            }

            model_total_time = time.time() - model_start_time
            print(f"✓ {model_type.upper()} completed in {model_total_time:.1f}s - Accuracy: {accuracy:.4f}")

            # Quick overfitting check
            if len(X_train) > 500:
                sample_size = min(500, len(X_train))
                train_acc = model_system.model.evaluate(X_train[:sample_size], y_train[:sample_size], verbose=0)[1]
                if train_acc - accuracy > 0.15:
                    print(f"⚠️  Overfitting detected (train: {train_acc:.3f}, test: {accuracy:.3f})")

        all_results[balancing_method] = method_results

    print(f"\nStep 6/6: Generating results...")
    step_start = time.time()

    # Results summary
    print(f"\n{'='*60}")
    print("RESULTS SUMMARY")
    print(f"{'='*60}")

    comparison_data = []
    for balancing_method in balancing_methods:
        for model_type in models_to_compare:
            result = all_results[balancing_method][model_type]
            comparison_data.append({
                'Balancing': balancing_method,
                'Model': model_type,
                'Accuracy': result['accuracy'],
                'Training Time (s)': result['training_time'],
                'Memory Usage (MB)': result['memory_usage']
            })

    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False, float_format='%.4f'))

    # Best combination
    best_idx = comparison_df['Accuracy'].idxmax()
    best_combination = comparison_df.iloc[best_idx]

    total_time = time.time() - total_start_time

    print(f"\n{'='*60}")
    print("BEST COMBINATION:")
    print(f"Balancing: {best_combination['Balancing']}")
    print(f"Model: {best_combination['Model']}")
    print(f"Accuracy: {best_combination['Accuracy']:.4f}")
    print(f"Total Runtime: {total_time:.1f} seconds ({total_time/60:.1f} minutes)")
    print(f"{'='*60}")

    print(f" Results generated in {time.time() - step_start:.1f}s")
    print(f" ANALYSIS COMPLETE! Total time: {total_time:.1f}s")

if __name__ == "__main__":
    compare_architectures()

ACTIVITY RECOGNITION - FAST MODE (30% DATASET)
Step 1/6: Loading dataset...
Dataset already exists, skipping download.
Loading accelerometry data from dataset...
Found 32 participant files
Loaded id079c763c.csv: 284300 samples
Loaded idf540d82b.csv: 243000 samples
Loaded idecc9265e.csv: 346600 samples
Loaded idc735fc09.csv: 345900 samples
Loaded id4ea159a8.csv: 267900 samples
Loaded idf1ce9a0f.csv: 328600 samples
Loaded id7c20ee7a.csv: 322300 samples
Loaded id34e056c8.csv: 341700 samples
Loaded id8af5374b.csv: 292700 samples
Loaded idbae5a811.csv: 422400 samples
Loaded id5308a7d6.csv: 323700 samples
Loaded idff99de96.csv: 330900 samples
Loaded id9603e9c3.csv: 111300 samples
Loaded id82b9735c.csv: 304800 samples
Loaded id650857ca.csv: 309000 samples
Loaded ida61e8ddf.csv: 123200 samples
Loaded idabd0c53c.csv: 94200 samples
Loaded id1c7e64ad.csv: 342700 samples
Loaded idc91a49d0.csv: 310100 samples
Loaded id1f372081.csv: 166100 samples
Loaded idfc5f05e4.csv: 285200 samples
Loaded id5993b